# architecture-29-02

## What's new:

1- Extract data rows with a formula instead of manually extracting them.


In [ ]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split


In [ ]:
# 1- Load and Scaling Features

df = pd.read_csv('XAGUSD-197001010000--H1-rates.csv', sep='\t')


# Select features to scale
features = ['<OPEN>', '<HIGH>', '<LOW>', '<CLOSE>', '<TICKVOL>']

# Apply MinMaxScaler
scaler = MinMaxScaler()
scaled = scaler.fit_transform(df[features])

In [ ]:
close_scaler = MinMaxScaler()
scaled_close = close_scaler.fit_transform(df[['<CLOSE>']])

In [ ]:
# Prepare sequences
def create_sequences(features, target, window, horizon):
    X, y = [], []
    for i in range(window, len(features) - horizon):
        X.append(features[i - window:i])
        y.append(target[i:i + horizon])
    return np.array(X), np.array(y)

X, y = create_sequences(scaled, scaled_close, 60, 10)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Build basic LSTM model
model = Sequential([
    LSTM(32, input_shape=(X_train.shape[1], X_train.shape[2])),
    Dense(10)
])
model.compile(optimizer='adam', loss='mse', metrics=['mae'])



In [ ]:
history = model.fit(
    X_train, y_train,
    epochs=50,
    verbose=1,
    validation_split=0.2,
)